### Import

In [28]:
import sys

sys.path.append('../')
sys.path.append('../ride/')

from ride_pfa import graph_osm_loader
import ride_pfa.centroid_graph.centroids_graph_builder as cgb
import ride_pfa.path_finding as pfa 
import ride_pfa.clustering as cls
import ride_pfa.utils as utils
import ride_pfa.benchmark as bench
from ride_pfa.centroid_graph import CentroidGraph

### Graph download

In [29]:
id = 'R13470549'
g = graph_osm_loader.get_graph(id, cache_path='./cache/')

\### Extraction Path Finding

In [30]:
# To decrease error in pathfinding better to use inverse weight in Louvain 
utils.add_inverse_edges_weight(g, weight='length', inverse_weight='inverse')

'inverse'

In [31]:
cms_resolver = cls.LouvainKMeansCommunityResolver(resolution=1, weight='inverse', k_means_weight='length', tqdm_progress_bar = True)
cg_builder = cgb.CentroidGraphBuilder()

def build_and_get_time() -> tuple[float, CentroidGraph]:
    return utils.get_execution_time(lambda: cg_builder.build(g, cms_resolver), iterations=1)

t, cg = build_and_get_time()
t, len(cg.cms)

k_means iterations:   0%|          | 0/20 [00:00<?, ?it/s]

find centroids:   0%|          | 0/26 [00:00<?, ?it/s]

find edges:   0%|          | 0/26 [00:00<?, ?it/s]

(2.341796398162842, 26)

In [32]:
baseline = pfa.Dijkstra(g)

In [33]:
test_alg = pfa.ExtractionPfa(
    g = g,
    upper=pfa.Dijkstra(cg.g),
    down=pfa.Dijkstra(g),
    cluster='cluster'
)

In [25]:
points = utils.generate_points(g, num=100)

In [26]:
stat = bench.PfaComparator(
    baseline=baseline,
    test_algorithm=test_alg,
    points=points,
    workers=4,
    with_tqdm_log = True
).compare()

In [27]:
stat


        err_max:    18.8231
        err_min:    0.0000
        err_median: 0.0000
        err_mean:   1.2900

        acceleration_max:    6.2156
        acceleration_min:    0.4652
        acceleration_median: 1.9199
        acceleration_mean:   2.0503
        